# Introduction

Jupiter notebook to run the simulations. For details about simulation algorithm, go in simulation.py file

There is two types of simulation:
1. Simulation with variable number of Animal in order ot find the Power at 80% for each infection model
2. Simulation with fix number of animals (found before for 80% power) but with different Threshold of sacrificed applied.
## Load data and packages

In [2]:
import simulation as sm
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('./data/df_for_analysis.xlsx',index_col=0)
df['Time_infection'] = df['Time_infection'].dt.strftime("%Y-%m-%d")
df['Time_point'] = df['Time_point'].apply(lambda x: pd.to_datetime(x.split(','),dayfirst=True,exact=False))

### prepare data for simulation

In [ ]:
#remove lethal dose
df = df[~df['Experiment'].str.contains("/LD")]

#add info of group control or not
df['control'] = df['exp'].apply(lambda x: 0 if x==0 else 1)

### columns used in simulations

In [ ]:
#list of column name for survival and time for each threshold
columns_time = df.columns[df.columns.str.contains('time')].values.tolist()
columns_survival = df.columns[df.columns.str.contains('survival')].values.tolist()

columns_survival = [columns_survival[-1]] + columns_survival[:-1] #PUT ORIGINAL DATA AT FIRST
columns_time = [columns_time[-1]] + columns_time[:-1] #PUT ORIGINAL DATA AT FIRST
column_threshold = list(zip(columns_survival,columns_time)) #USEFUL TO LOOP OVER CONDITION: THRESHOLD

### number of Mice used for the simulation 

In [ ]:
#number of mice per group
mice_100 = np.logspace(1, 2, num=20).astype(int).tolist() #log scale until N = 100
mice_200 = np.arange(110,210,10).astype(int).tolist() # linear scale 100-200
mice_400 = np.arange(210,410,10).astype(int).tolist() # linear scale 200-400 not alwasy used if simulation converge
mice_range = mice_100 + mice_200 + mice_400

## Simulation
### with raw data (no threshold apply) with varying number of mice

In [ ]:
sm.automatic_simulation_per_infection(df,
N_simulation=5000,
N_repetition=20,
mice_range = mice_range,
cpu=11)

### with fix number of mice and varying Threshold

In [ ]:
infection_of_interest = ["Listeria","S. pneumoniae"]
N_mice_80 = [35,56]
dict_mice_range = dict(zip(infection_of_interest,N_mice_80))
df = df[df['Infection'].isin(infection_of_interest)]

In [ ]:
sm.automatic_simulation_per_infection_fix_mice(df,
N_simulation=5000,
N_repetition=20,
column_threshold=column_threshold,
dict_mice_range = dict_mice_range,
cpu=11)